In [1]:
%pip install darts --upgrade
!pip install lightgbm
from darts import TimeSeries
from darts.models import LightGBMModel
import warnings
import lightgbm as lgb
warnings.filterwarnings("ignore")
import pandas as pd
import datetime

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Step 1: Load and Clean the Data
data = pd.read_excel('final_merged.xlsx')

In [3]:
min_date = data['Year'].min()
max_date = data['Year'].max()

print('The date range for this data is ' + str(min_date) + ' to ' + str(max_date))

The date range for this data is 1990 to 2022


In [4]:
num_countries = data['Country'].nunique()

print('The number of countries in this data is ' + str(num_countries))

The number of countries in this data is 195


In [5]:
data = data[(data['Year'] >= 1990) & (data['Year'] <= 2022)]

In [6]:
data = data.pivot_table(values='Human Development Index (value)', index='Year', columns='Country')
data.index = data.index.astype(int)
data = data.rename_axis(columns = None)
data.reset_index(inplace=True)
data['Year'] = pd.to_datetime(data['Year'], format='%Y')

In [7]:
data

,Year,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,...,United Kingdom,United States,Uruguay,Uzbekistan,Vanuatu,Venezuela (Bolivarian Republic of),Viet Nam,Yemen,Zambia,Zimbabwe
0,1990-01-01,0.284,0.649,0.593,0.000,0.000,0.000,0.724,0.658,0.864,...,0.804,0.875,0.702,0.000,0.000,0.657,0.492,0.357,0.417,0.479
1,1991-01-01,0.292,0.632,0.596,0.000,0.000,0.000,0.731,0.646,0.866,...,0.809,0.875,0.706,0.000,0.000,0.668,0.503,0.361,0.411,0.481
2,1992-01-01,0.299,0.616,0.601,0.000,0.000,0.000,0.736,0.611,0.868,...,0.815,0.880,0.709,0.000,0.000,0.675,0.514,0.368,0.408,0.465
3,1993-01-01,0.307,0.618,0.602,0.000,0.000,0.000,0.740,0.613,0.873,...,0.819,0.882,0.712,0.000,0.000,0.678,0.526,0.374,0.412,0.460
4,1994-01-01,0.300,0.623,0.603,0.000,0.000,0.000,0.746,0.617,0.873,...,0.827,0.885,0.718,0.000,0.000,0.679,0.536,0.378,0.407,0.457
5,1995-01-01,0.318,0.632,0.607,0.000,0.000,0.000,0.747,0.623,0.882,...,0.830,0.886,0.720,0.000,0.000,0.685,0.550,0.389,0.407,0.449
6,1996-01-01,0.326,0.644,0.618,0.000,0.000,0.000,0.752,0.627,0.884,...,0.836,0.887,0.726,0.000,0.000,0.687,0.560,0.400,0.409,0.448
7,1997-01-01,0.330,0.641,0.626,0.000,0.000,0.000,0.758,0.635,0.887,...,0.843,0.889,0.738,0.000,0.000,0.694,0.571,0.408,0.410,0.445
8,1998-01-01,0.329,0.656,0.635,0.000,0.000,0.000,0.764,0.647,0.891,...,0.848,0.891,0.747,0.000,0.000,0.697,0.580,0.417,0.409,0.439
9,1999-01-01,0.337,0.668,0.644,0.000,0.369,0.000,0.774,0.651,0.894,...,0.855,0.891,0.751,0.000,0.000,0.695,0.589,0.424,0.412,0.431


In [8]:
import pandas as pd
import numpy as np
import warnings
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima

# Ignore warnings
warnings.filterwarnings("ignore")

# Step 1: Load and Clean the Data
data = pd.read_excel('final_merged.xlsx')

# Filter the data for the years 1990 to 2022 and pivot it
data = data[(data['Year'] >= 1990) & (data['Year'] <= 2022)]
data = data.pivot(index='Year', columns='Country', values='Human Development Index (value)')

# Check if any country has missing values
print(data.isnull().sum())

# Fill missing values (if any) using forward fill
data.fillna(method='ffill', inplace=True)

# Step 2: Forecasting for each country using Auto ARIMA
predictions = {}

# Number of years to predict
n_years = 10

for country in data.columns:
    country_data = data[country].dropna()
    if len(country_data) < 2:
        print(f"Not enough data for {country}")
        continue

    # Fit an Auto ARIMA model
    model = auto_arima(country_data, seasonal=False, stepwise=True)
    print(f"Optimal model order for {country}: {model.order}")
    
    # Fit the model
    model_fit = model.fit(country_data)

    # Make predictions for the next 10 years
    forecast = model_fit.predict(n_periods=n_years)
    predictions[country] = forecast

# Step 3: Prepare DataFrame with predictions and corresponding years
years = list(range(2023, 2023 + n_years))  # Generate years for predictions
if predictions:
    predictions_df = pd.DataFrame(predictions, index=years)

    # Save predictions to a CSV file
    predictions_df.to_csv('HDI_predictions_ARIMA_10_years.csv', index=True)

    # Step 4: Display Predictions
    print(predictions_df)
else:
    print("No predictions were made.")

Country
Afghanistan                           0
Albania                               0
Algeria                               0
Andorra                               0
Angola                                0
                                     ..
Venezuela (Bolivarian Republic of)    0
Viet Nam                              0
Yemen                                 0
Zambia                                0
Zimbabwe                              0
Length: 195, dtype: int64
Optimal model order for Afghanistan: (1, 1, 0)
Optimal model order for Albania: (1, 1, 0)
Optimal model order for Algeria: (0, 1, 0)
Optimal model order for Andorra: (0, 1, 0)
Optimal model order for Angola: (0, 1, 0)
Optimal model order for Antigua and Barbuda: (0, 1, 0)
Optimal model order for Argentina: (0, 2, 1)
Optimal model order for Armenia: (0, 1, 1)
Optimal model order for Australia: (1, 2, 1)
Optimal model order for Austria: (3, 2, 2)
Optimal model order for Azerbaijan: (0, 1, 0)
Optimal model order for Bahamas

Optimal model order for Syrian Arab Republic: (1, 0, 1)
Optimal model order for Tajikistan: (1, 1, 0)
Optimal model order for Tanzania (United Republic of): (1, 1, 0)
Optimal model order for Thailand: (0, 1, 0)
Optimal model order for Timor-Leste: (0, 1, 0)
Optimal model order for Togo: (1, 1, 0)
Optimal model order for Tonga: (0, 1, 1)
Optimal model order for Trinidad and Tobago: (0, 1, 0)
Optimal model order for Tunisia: (1, 2, 1)
Optimal model order for Turkey: (0, 1, 0)
Optimal model order for Turkmenistan: (0, 1, 0)
Optimal model order for Tuvalu: (0, 1, 1)
Optimal model order for Uganda: (1, 1, 0)
Optimal model order for Ukraine: (1, 1, 0)
Optimal model order for United Arab Emirates: (1, 1, 0)
Optimal model order for United Kingdom: (0, 1, 0)
Optimal model order for United States: (0, 1, 0)
Optimal model order for Uruguay: (0, 1, 0)
Optimal model order for Uzbekistan: (0, 1, 0)
Optimal model order for Vanuatu: (0, 1, 0)
Optimal model order for Venezuela (Bolivarian Republic of):

In [9]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Assume `train` and `test` are your training and testing datasets
# Fit ARIMA model on training data
model = ARIMA(train, order=(1, 1, 1))
model_fit = model.fit()

# Predict on test data
predictions = model_fit.forecast(steps=len(test))

# Calculate evaluation metrics
mae = mean_absolute_error(test, predictions)
mse = mean_squared_error(test, predictions)
rmse = np.sqrt(mse)

print(f"MAE: {mae}, MSE: {mse}, RMSE: {rmse}")

# Plotting
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(test.index, test, label='Actual')
plt.plot(test.index, predictions, label='Predicted', color='red')
plt.title('Actual vs Predicted')
plt.legend()
plt.show()

NameError: name 'train' is not defined